Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-data-dependency-steps.png)

# Azure Machine Learning Data Prep
In this notebook, we will see how we can do data prep using the azureml.dataprep SDK.

### Azure Machine Learning and Pipeline SDK-specific Imports

In [1]:
import azureml.core
import azureml.dataprep
from azureml.core import Workspace, Experiment, Datastore, Dataset
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.widgets import RunDetails
from azureml.data import TabularDataset
from azureml.data.datapath import DataPath
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep

import azureml.dataprep as dprep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.83


### Initialize Workspace and Retrieve a Compute Target

In [2]:
ws = Workspace.from_config()
print("== Workspace:")
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# Default datastore (Azure blob storage)
# def_blob_store = ws.get_default_datastore()
blob_store = Datastore(ws, "workspaceblobstore")
print("== Datastore: {}".format(blob_store.name))

# list compute targets
print("== Compute targets:")
for ct in ws.compute_targets:
    print("  " + ct)
    
# Retrieve a compute target    
from azureml.core.compute_target import ComputeTargetException
aml_compute_target = "agd-training-cpu"
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("== AML compute target attached: " + aml_compute_target)
except ComputeTargetException:
    print("== AML compute target not found: " + aml_compute_target)

== Workspace:
agd-mlws
azure-ml-workshop
westus2
c5ec24ce-9c5f-4da2-bf12-9ca8e9758d60
== Datastore: workspaceblobstore
== Compute targets:
  agd-inference
  agd-inference-v
  agd-training-gpu
  agd-training-cpu
== AML compute target attached: agd-training-cpu


### Create Compute Configuration

This step uses a docker image, use a [**RunConfiguration**](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.runconfiguration?view=azure-ml-py) to specify these requirements and use when creating the PythonScriptStep. 

In [3]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.core.conda_dependencies import CondaDependencies

# create a new runconfig object
run_config = RunConfiguration()

# enable Docker 
run_config.environment.docker.enabled = True

# set Docker base image to the default CPU-based image
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE

# use conda_dependencies.yml to create a conda environment in the Docker image for execution
run_config.environment.python.user_managed_dependencies = False

# specify dependencies
#run_config.environment.python.conda_dependencies = CondaDependencies.create(
#    conda_packages=['pandas'],
#    pip_packages=['azureml-sdk', 'azureml-dataprep[fuse,pandas]', 'azureml-train-automl'], 
#    pin_sdk_version=False)
run_config.environment.python.conda_dependencies = CondaDependencies(
    conda_dependencies_file_path='data-prep-pipeline.yml')

#
print("== Run Configuration created")

== Run Configuration created


## Building Pipeline Steps with Inputs and Outputs
As mentioned earlier, a step in the pipeline can take data as input. This data can be a data source that lives in one of the accessible data locations, or intermediate data produced by a previous step in the pipeline.

### Data inputs / output definitions

In [4]:
# inputs

# output
d_use_case_1_dataprep_sdk_pd = PipelineData("d_use_case_1_dataprep_sdk",datastore=blob_store)

print("== Datasets defined")

== Datasets defined


In [110]:
# In notebook development of the dataprep.py script
h_ts_1_dflow = dprep.read_csv(DataPath(blob_store,'datasets/time-series/S_ACTUALS.csv'))
h_ts_2_dflow = dprep.read_csv(DataPath(blob_store,'datasets/time-series/W_ACTUALS.csv'))
h_ts_3_dflow = dprep.read_csv(DataPath(blob_store,'datasets/time-series/C_LOAD.csv'))
d_ts_1_dflow = dprep.read_csv(DataPath(blob_store,'datasets/time-series/X1.csv'))
d_ts_2_dflow = dprep.read_csv(DataPath(blob_store,'datasets/time-series/X2.csv'))

In [111]:
h_ts_1_dflow.head(6)

,MYDATE,HOUR,NODE_ID,MW
0,1/1/2012,1,N1,2.78994586
1,1/1/2012,1,S1,-3.860853503
2,1/1/2012,1,Z1,-0.675159236
3,1/1/2012,2,N1,2.785633758
4,1/1/2012,2,S1,-3.788318471
5,1/1/2012,2,Z1,-0.675159236


In [112]:
h_ts_1_dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
MYDATE,FieldType.STRING,1/1/2012,9/9/2015,100799.0,0.0,100799.0,0.0,0.0,0.0,>1000,,,,,,,,,,,,,,,,
HOUR,FieldType.STRING,1,9,100799.0,0.0,100799.0,0.0,0.0,0.0,25,,,,,,,,,,,,,,,,
NODE_ID,FieldType.STRING,N1,Z1,100799.0,0.0,100799.0,0.0,0.0,0.0,3,,,,,,,,,,,,,,,,
MW,FieldType.STRING,-0.00033121,999.9626592,100799.0,0.0,100799.0,0.0,0.0,0.0,>1000,,,,,,,,,,,,,,,,


In [113]:
# discover data types
builder = h_ts_1_dflow.builders.set_column_types()
builder.learn()
builder.conversion_candidates

{'MYDATE': [
     (FieldType.DATE, ['%d/%m/%Y']),
     (FieldType.DATE, ['%m/%d/%Y'])],
 'HOUR': [FieldType.INTEGER],
 'NODE_ID': [FieldType.STRING],
 'MW': [FieldType.DECIMAL]}

In [114]:
# resolve MYDATE type ambiguity
builder.conversion_candidates['MYDATE'] = (dprep.FieldType.DATE, ['%m/%d/%Y'])
h_ts_1_dflow = builder.to_dataflow()
h_ts_1_dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
MYDATE,FieldType.DATE,2012-01-01 00:00:00+00:00,2016-02-29 00:00:00+00:00,100799.0,0.0,100799.0,0.0,0.0,0.0,>1000,,,,,,,,,,,,,,,,
HOUR,FieldType.INTEGER,1,25,100799.0,0.0,100799.0,0.0,0.0,0.0,25,1,3,3,6.28018,12.0127,18.0145,23,24,24,12.4607,6.90726,47.7102,0.0036578,-1.20297,25,1
NODE_ID,FieldType.STRING,N1,Z1,100799.0,0.0,100799.0,0.0,0.0,0.0,3,,,,,,,,,,,,,,,,
MW,FieldType.DECIMAL,-32.7938,2437.81,100799.0,0.0,100799.0,0.0,0.0,0.0,>1000,-10.5711,-3.65979,-3.76832,-1.14616,7.83233,314.028,1608.31,2131.88,2370.15,274.534,506.661,256705,2.3246,4.63076,669.44,-32.7938


In [172]:
# instead 'auto_read_file' will do type auto detect (when it can), and other things (auto skip first rows, will find the right header, etc)
h_ts_1_dflow = dprep.auto_read_file(DataPath(blob_store,'datasets/time-series/S_ACTUALS.csv'))
h_ts_1_dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
MYDATE,FieldType.STRING,1/1/2012,9/9/2015,100799.0,0.0,100799.0,0.0,0.0,0.0,>1000,,,,,,,,,,,,,,,,
HOUR,FieldType.INTEGER,1,25,100799.0,0.0,100799.0,0.0,0.0,0.0,25,1,3,3,6.28018,12.0127,18.0145,23,24,24,12.4607,6.90726,47.7102,0.0036578,-1.20297,25,1
NODE_ID,FieldType.STRING,N1,Z1,100799.0,0.0,100799.0,0.0,0.0,0.0,3,,,,,,,,,,,,,,,,
MW,FieldType.DECIMAL,-32.7938,2437.81,100799.0,0.0,100799.0,0.0,0.0,0.0,>1000,-10.5711,-3.65979,-3.76832,-1.14616,7.83233,314.028,1608.31,2131.88,2370.15,274.534,506.661,256705,2.3246,4.63076,669.44,-32.7938


In [173]:
# =================
# dataprep.py (WIP)
# =================

h_ts_1_dflow = dprep.auto_read_file(DataPath(blob_store,'datasets/time-series/S_ACTUALS.csv'))
h_ts_2_dflow = dprep.auto_read_file(DataPath(blob_store,'datasets/time-series/W_ACTUALS.csv'))
h_ts_3_dflow = dprep.auto_read_file(DataPath(blob_store,'datasets/time-series/C_LOAD.csv'))
d_ts_1_dflow = dprep.auto_read_file(DataPath(blob_store,'datasets/time-series/X1.csv'))
d_ts_2_dflow = dprep.auto_read_file(DataPath(blob_store,'datasets/time-series/X2.csv'))

# =============================
# end of dataprep.py code (WIP)
# =============================

In [176]:
# ts_1 and ts_2 need to be pivoted
h_ts_1_dflow.head(6)

,MYDATE,HOUR,NODE_ID,MW
0,1/1/2012,1,N1,2.789946
1,1/1/2012,1,S1,-3.860854
2,1/1/2012,1,Z1,-0.675159
3,1/1/2012,2,N1,2.785634
4,1/1/2012,2,S1,-3.788318
5,1/1/2012,2,Z1,-0.675159


In [177]:
# PIVOT data
h_ts_1_pivot_dflow = h_ts_1_dflow.pivot(['NODE_ID'],'MW',
                                        azureml.dataprep.api.engineapi.typedefinitions.SummaryFunction.MAX,
                                        ['MYDATE','HOUR'])
h_ts_1_pivot_dflow.head(2)

,MYDATE,HOUR,N1,S1,Z1
0,1/1/2012,1,2.789946,-3.860854,-0.675159
1,1/1/2012,2,2.785634,-3.788318,-0.675159


In [178]:
# using derive_column_by_example to reformat DATE/HOUR as a DATETIME field
builder = h_ts_1_pivot_dflow.builders.derive_column_by_example(source_columns = ['MYDATE','HOUR'], new_column_name = 'DATETIME')
builder.add_example(source_data = {'MYDATE': '1/1/2012', 'HOUR': 1}, example_value = '01/01/2012 01:00')
builder.add_example(source_data = {'MYDATE': '10/10/2012', 'HOUR': 15}, example_value = '10/10/2012 15:00')
builder.add_example(source_data = {'MYDATE': '1/17/2012', 'HOUR': 12}, example_value = '01/17/2012 12:00')
builder.preview(skip=3000,count=20)

,MYDATE,HOUR,DATETIME
0,5/14/2012,7,05/14/2012 07:00
1,5/14/2012,8,05/14/2012 08:00
2,5/14/2012,9,05/14/2012 09:00
3,5/14/2012,10,05/14/2012 10:00
4,5/14/2012,11,05/14/2012 11:00
5,5/14/2012,12,05/14/2012 12:00
6,5/14/2012,13,05/14/2012 13:00
7,5/14/2012,14,05/14/2012 14:00
8,5/14/2012,15,05/14/2012 15:00
9,5/14/2012,16,05/14/2012 16:00


In [179]:
h_ts_1_pivot_dt_dflow = builder.to_dataflow()
h_ts_1_pivot_dt_dflow.head(6)

,MYDATE,HOUR,DATETIME,N1,S1,Z1
0,1/1/2012,1,01/01/2012 01:00,2.789946,-3.860854,-0.675159
1,1/1/2012,2,01/01/2012 02:00,2.785634,-3.788318,-0.675159
2,1/1/2012,3,01/01/2012 03:00,2.786000,-3.887990,-0.676115
3,1/1/2012,4,01/01/2012 04:00,2.785965,-4.119672,-0.700669
4,1/1/2012,5,01/01/2012 05:00,2.790325,-4.828436,-0.702354
5,1/1/2012,6,01/01/2012 06:00,2.788650,-5.092978,-0.706051


In [180]:
h_ts_1_pivot_dt_dflow.dtypes

MYDATE                   FieldType.STRING
HOUR                     FieldType.INTEGER
DATETIME                 FieldType.STRING
N1                       FieldType.DECIMAL
S1                       FieldType.DECIMAL
Z1                       FieldType.DECIMAL

In [146]:
h_ts_1_pivot_dt_dflow = h_ts_1_pivot_dflow.derive_column_by_example(
    source_columns = ['MYDATE','HOUR'],
    new_column_name = 'DATETIME',
    example_data = [({'MYDATE': '1/1/2012', 'HOUR': '1'},    '01/01/2012 01:00'),
                    ({'MYDATE': '10/10/2012', 'HOUR': '15'}, '10/10/2012 15:00'),
                    ({'MYDATE': '1/17/2012', 'HOUR': '12'},  '01/17/2012 12:00')]
    ).drop_columns(['MYDATE','HOUR'])
h_ts_1_pivot_dt_dflow.head(4)

,DATETIME,N1,S1,Z1
0,01/01/2012 01:00,2.789946,-3.860854,-0.675159
1,01/01/2012 02:00,2.785634,-3.788318,-0.675159
2,01/01/2012 03:00,2.786000,-3.887990,-0.676115
3,01/01/2012 04:00,2.785965,-4.119672,-0.700669


In [151]:
builder = h_ts_1_pivot_dt_dflow.builders.set_column_types()
builder.learn()
builder.conversion_candidates

{'DATETIME': [
     (FieldType.DATE, ['%d/%m/%Y %H:%M']),
     (FieldType.DATE, ['%m/%d/%Y %H:%M'])],
 'N1': [FieldType.DECIMAL],
 'S1': [FieldType.DECIMAL],
 'Z1': [FieldType.DECIMAL]}

In [154]:
builder.conversion_candidates['DATETIME'] = (dprep.FieldType.DATE, ['%d/%m/%Y %H:%M'])
h_ts_1_pivot_dt_dflow = builder.to_dataflow()
h_ts_1_pivot_dt_dflow.head(4)

,DATETIME,N1,S1,Z1
0,2012-01-01 01:00:00,2.789946,-3.860854,-0.675159
1,2012-01-01 02:00:00,2.785634,-3.788318,-0.675159
2,2012-01-01 03:00:00,2.786000,-3.887990,-0.676115
3,2012-01-01 04:00:00,2.785965,-4.119672,-0.700669


In [155]:
h_ts_1_pivot_dt_dflow.dtypes

DATETIME                 FieldType.DATE  
N1                       FieldType.DECIMAL
S1                       FieldType.DECIMAL
Z1                       FieldType.DECIMAL

In [156]:
h_ts_1_pivot_dt_dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
DATETIME,FieldType.DATE,2012-01-01 01:00:00+00:00,2016-12-02 23:00:00+00:00,33600.0,4.0,33596.0,0.000119,20391.0,0.0,>1000,,,,,,,,,,,,,,,,
N1,FieldType.DECIMAL,-32.7938,473.713,33600.0,0.0,33600.0,0.000000,0.0,0.0,>1000,-7.02568,-1.00306,-1.00727,-0.738948,5.28022,196.082,391.086,433.924,452.942,102.114,137.342,18862.9,1.0419,-0.32315,473.713,-32.7938
S1,FieldType.DECIMAL,-14.5467,2437.81,33600.0,0.0,33600.0,0.000000,0.0,0.0,>1000,-10.9195,-7.45157,-7.49409,-5.27086,27.9864,1280.7,2024.96,2282.61,2396.81,592.049,755.079,570144,0.825234,-0.860473,2437.81,-14.5467
Z1,FieldType.DECIMAL,-2.30379,563.445,33600.0,1.0,33599.0,0.000030,0.0,0.0,>1000,-2.11192,-1.61922,-1.62392,-1.35039,4.72356,270.328,468.624,522.22,553.707,129.436,171.8,29515.2,0.93803,-0.638352,563.445,-2.30379


In [185]:
# =================
# dataprep.py (WIP)
# =================

h_ts_1_dflow = dprep.auto_read_file(DataPath(blob_store,'datasets/time-series/S_ACTUALS.csv'))
h_ts_2_dflow = dprep.auto_read_file(DataPath(blob_store,'datasets/time-series/W_ACTUALS.csv'))
h_ts_3_dflow = dprep.auto_read_file(DataPath(blob_store,'datasets/time-series/C_LOAD.csv'))
d_ts_1_dflow = dprep.auto_read_file(DataPath(blob_store,'datasets/time-series/X1.csv'))
d_ts_2_dflow = dprep.auto_read_file(DataPath(blob_store,'datasets/time-series/X2.csv'))

# Pivot data
h_ts_1_pivot_dflow = h_ts_1_dflow.pivot(['NODE_ID'],'MW',
                                        azureml.dataprep.api.engineapi.typedefinitions.SummaryFunction.MAX,
                                        ['MYDATE','HOUR'])
h_ts_2_pivot_dflow = h_ts_2_dflow.pivot(['NODE_ID'],'MW',
                                        azureml.dataprep.api.engineapi.typedefinitions.SummaryFunction.MAX,
                                        ['MYDATE','HOUR'])

# merge DATE and HOUR and update datatype for DATETIME column
def ts_merge_date_hour_to_datetime(dflow, date_column_name, hour_column_name):
    # merge columns
    dflow = dflow.derive_column_by_example(
        source_columns = [date_column_name,hour_column_name],
        new_column_name = 'DATETIME',
        example_data = [({date_column_name: '1/1/2012', hour_column_name: '1'},    '01/01/2012 01:00'),
                        ({date_column_name: '10/10/2012', hour_column_name: '15'}, '10/10/2012 15:00'),
                        ({date_column_name: '1/17/2012', hour_column_name: '12'},  '01/17/2012 12:00')]
        ).drop_columns([date_column_name,hour_column_name])
    # update data type
    builder = dflow.builders.set_column_types()
    builder.learn()
    builder.conversion_candidates['DATETIME'] = (dprep.FieldType.DATE, ['%d/%m/%Y %H:%M'])
    return builder.to_dataflow()

# generate all DATETIME columns with proper data type
h_ts_1_pivot_dt_dflow = ts_merge_date_hour_to_datetime(h_ts_1_pivot_dflow,'MYDATE','HOUR')
h_ts_2_pivot_dt_dflow = ts_merge_date_hour_to_datetime(h_ts_2_pivot_dflow,'MYDATE','HOUR')
h_ts_3_dt_dflow =       ts_merge_date_hour_to_datetime(h_ts_3_dflow,      'DATE','HE')

# =============================
# end of dataprep.py code (WIP)
# =============================

In [186]:
# check profile
h_ts_1_pivot_dt_dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
DATETIME,FieldType.DATE,2012-01-01 01:00:00+00:00,2016-12-02 23:00:00+00:00,33600.0,4.0,33596.0,0.000119,20391.0,0.0,>1000,,,,,,,,,,,,,,,,
N1,FieldType.DECIMAL,-32.7938,473.713,33600.0,0.0,33600.0,0.000000,0.0,0.0,>1000,-7.02568,-1.00306,-1.00727,-0.738948,5.28022,196.082,391.086,433.924,452.942,102.114,137.342,18862.9,1.0419,-0.32315,473.713,-32.7938
S1,FieldType.DECIMAL,-14.5467,2437.81,33600.0,0.0,33600.0,0.000000,0.0,0.0,>1000,-10.9195,-7.45157,-7.49409,-5.27086,27.9864,1280.7,2024.96,2282.61,2396.81,592.049,755.079,570144,0.825234,-0.860473,2437.81,-14.5467
Z1,FieldType.DECIMAL,-2.30379,563.445,33600.0,1.0,33599.0,0.000030,0.0,0.0,>1000,-2.11192,-1.61922,-1.62392,-1.35039,4.72356,270.328,468.624,522.22,553.707,129.436,171.8,29515.2,0.93803,-0.638352,563.445,-2.30379


In [187]:
# check profile
d_ts_1_dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
RDATE,FieldType.DATE,2012-01-01 00:00:00+00:00,2016-02-29 00:00:00+00:00,1521.0,0.0,1521.0,0.0,0.0,0.0,>1000,,,,,,,,,,,,,,,,
X1,FieldType.DECIMAL,50.4,106.8,1521.0,0.0,1521.0,0.0,0.0,0.0,245,50.6462,62.1737,62.0222,69.172,78.166,88.0893,95.7833,100.633,105.983,78.1219,11.5587,133.604,-0.0766716,-0.901754,106.8,50.4


In [189]:
# JOINING DATA SET h_ts_j1=h1,h2
h_ts_j1_dflow = dprep.Dataflow.join(
    h_ts_1_pivot_dt_dflow,
    h_ts_2_pivot_dt_dflow,
    join_key_pairs=[('DATETIME', 'DATETIME')],
    left_column_prefix='h1_',
    right_column_prefix='h2_'
).drop_columns('h2_DATETIME').rename_columns({'h1_DATETIME':'DATETIME'})
h_ts_j1_dflow.head(5)

,DATETIME,h1_N1,h1_S1,h1_Z1,h2_N1,h2_S1
0,2012-01-01 01:00:00,2.789946,-3.860854,-0.675159,214.201076,76.939389
1,2012-01-01 02:00:00,2.785634,-3.788318,-0.675159,208.534389,66.742051
2,2012-01-01 03:00:00,2.786000,-3.887990,-0.676115,188.653331,70.460099
3,2012-01-01 04:00:00,2.785965,-4.119672,-0.700669,192.039484,76.141580
4,2012-01-01 05:00:00,2.790325,-4.828436,-0.702354,204.381194,79.607589


In [190]:
# JOINING DATA SET h_ts_dflow=h_ts_j1,h3
h_ts_dflow = dprep.Dataflow.join(
    h_ts_j1_dflow,
    h_ts_3_dt_dflow,
    join_key_pairs=[('DATETIME', 'DATETIME')],
    left_column_prefix='',
    right_column_prefix='h3_'
).drop_columns('h3_DATETIME')
h_ts_dflow.head(5)

,DATETIME,h1_N1,h1_S1,h1_Z1,h2_N1,h2_S1,h3_CLOAD
0,2012-01-01 01:00:00,2.789946,-3.860854,-0.675159,214.201076,76.939389,7458.280255
1,2012-01-01 02:00:00,2.785634,-3.788318,-0.675159,208.534389,66.742051,7214.649682
2,2012-01-01 03:00:00,2.786000,-3.887990,-0.676115,188.653331,70.460099,7018.789809
3,2012-01-01 04:00:00,2.785965,-4.119672,-0.700669,192.039484,76.141580,6912.420382
4,2012-01-01 05:00:00,2.790325,-4.828436,-0.702354,204.381194,79.607589,6913.057325


In [268]:
# =================
# dataprep.py (WIP)
# =================

h_ts_1_dflow = dprep.auto_read_file(DataPath(blob_store,'datasets/time-series/S_ACTUALS.csv'))
h_ts_2_dflow = dprep.auto_read_file(DataPath(blob_store,'datasets/time-series/W_ACTUALS.csv'))
h_ts_3_dflow = dprep.auto_read_file(DataPath(blob_store,'datasets/time-series/C_LOAD.csv'))
d_ts_1_dflow = dprep.auto_read_file(DataPath(blob_store,'datasets/time-series/X1.csv'))
d_ts_2_dflow = dprep.auto_read_file(DataPath(blob_store,'datasets/time-series/X2.csv'))

# Pivot data
h_ts_1_pivot_dflow = h_ts_1_dflow.pivot(['NODE_ID'],'MW',
                                        azureml.dataprep.api.engineapi.typedefinitions.SummaryFunction.MAX,
                                        ['MYDATE','HOUR'])
h_ts_2_pivot_dflow = h_ts_2_dflow.pivot(['NODE_ID'],'MW',
                                        azureml.dataprep.api.engineapi.typedefinitions.SummaryFunction.MAX,
                                        ['MYDATE','HOUR'])

# merge DATE and HOUR and update datatype for DATETIME column
def ts_merge_date_hour_to_datetime(dflow, date_column_name, hour_column_name):
    # merge columns
    dflow = dflow.derive_column_by_example(
        source_columns = [date_column_name,hour_column_name],
        new_column_name = 'DATETIME',
        example_data = [({date_column_name: '1/1/2012', hour_column_name: '1'},    '01/01/2012 01:00'),
                        ({date_column_name: '10/10/2012', hour_column_name: '15'}, '10/10/2012 15:00'),
                        ({date_column_name: '1/17/2012', hour_column_name: '12'},  '01/17/2012 12:00')]
        ).drop_columns([date_column_name,hour_column_name])
    # update data type
    builder = dflow.builders.set_column_types()
    builder.learn()
    builder.conversion_candidates['DATETIME'] = (dprep.FieldType.DATE, ['%d/%m/%Y %H:%M'])
    return builder.to_dataflow()

# generate all DATETIME columns with proper data type
h_ts_1_pivot_dt_dflow = ts_merge_date_hour_to_datetime(h_ts_1_pivot_dflow,'MYDATE','HOUR')
h_ts_2_pivot_dt_dflow = ts_merge_date_hour_to_datetime(h_ts_2_pivot_dflow,'MYDATE','HOUR')
h_ts_3_dt_dflow =       ts_merge_date_hour_to_datetime(h_ts_3_dflow,      'DATE','HE')

# JOINING DATA SET h_ts_dflow=join(h1,h2,h3)
h_ts_dflow = dprep.Dataflow.join(
    dprep.Dataflow.join(h_ts_1_pivot_dt_dflow,
                        h_ts_2_pivot_dt_dflow,
                        join_key_pairs=[('DATETIME', 'DATETIME')],
                        left_column_prefix='h1_',right_column_prefix='h2_'
                       ).drop_columns(['h2_DATETIME']).rename_columns({'h1_DATETIME':'DATETIME'}),
    h_ts_3_dt_dflow,
    join_key_pairs=[('DATETIME', 'DATETIME')],
    left_column_prefix='',
    right_column_prefix='h3_'
).drop_columns('h3_DATETIME')

# JOINING DATA SET d_ts_dflow=join(d1,d2)
d_ts_dflow = dprep.Dataflow.join(
    d_ts_1_dt_dflow,
    d_ts_2_dt_dflow,
    join_key_pairs=[('RDATE', 'RDATE')],
    left_column_prefix='',
    right_column_prefix='r_').drop_columns(['r_RDATE']).rename_columns({'r_X2':'X2'})

# =============================
# end of dataprep.py code (WIP)
# =============================

In [269]:
# check profile 
h_ts_dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
DATETIME,FieldType.DATE,2012-01-01 01:00:00+00:00,2016-12-02 23:00:00+00:00,33653.0,64.0,33589.0,0.001902,20386.0,0.0,>1000,,,,,,,,,,,,,,,,
h1_N1,FieldType.DECIMAL,-32.7938,473.713,33653.0,0.0,33653.0,0.000000,0.0,0.0,>1000,-7.01141,-1.00289,-1.0049,-0.738381,4.7042,196.121,390.774,433.882,452.929,101.897,137.272,18843.7,1.04504,-0.316437,473.713,-32.7938
h1_S1,FieldType.DECIMAL,-14.5467,2437.81,33653.0,0.0,33653.0,0.000000,0.0,0.0,>1000,-10.9219,-7.45055,-7.49119,-5.2657,26.8337,1279.15,2026.29,2281.04,2396.56,590.765,754.705,569580,0.828503,-0.855033,2437.81,-14.5467
h1_Z1,FieldType.DECIMAL,-2.30379,563.445,33653.0,1.0,33652.0,0.000030,0.0,0.0,>1000,-2.11183,-1.6188,-1.62755,-1.34989,4.5957,269.651,468.544,521.809,554.027,129.157,171.709,29483.8,0.941151,-0.632476,563.445,-2.30379
h2_N1,FieldType.DECIMAL,-3.65543,458.068,33653.0,0.0,33653.0,0.000000,0.0,0.0,>1000,-2.7496,1.67475,1.5734,17.3363,81.8513,219.794,363.396,413.888,438.416,125.987,123.462,15242.9,0.770255,-0.67567,458.068,-3.65543
h2_S1,FieldType.DECIMAL,-5.92685,1143.99,33653.0,0.0,33653.0,0.000000,0.0,0.0,>1000,-3.94448,26.6717,25.9997,70.8251,244.402,521.081,814.267,968.584,1078.06,315.286,268.397,72037,0.669233,-0.646651,1143.99,-5.92685
h3_CLOAD,FieldType.DECIMAL,4950.32,15894.6,33653.0,0.0,33653.0,0.000000,0.0,0.0,>1000,5771.88,6541.17,6529.08,7076.07,7948.11,8787.34,11415.6,13170.1,14450.9,8178.58,1518.85,2.3069e+06,1.29312,1.95259,11309.6,4950.32


In [278]:
# TODO ==> KEEP DATE COLUMN IN H to properly summarize by DATE and not DATETIME in H
# TODO ==> KEEP DATE COLUMN IN H to properly summarize by DATE and not DATETIME in H
# TODO ==> KEEP DATE COLUMN IN H to properly summarize by DATE and not DATETIME in H
# TODO ==> KEEP DATE COLUMN IN H to properly summarize by DATE and not DATETIME in H
# TODO ==> KEEP DATE COLUMN IN H to properly summarize by DATE and not DATETIME in H

In [274]:
# summarize h_ts_dflow to daily
def generate_summary_column(column_name,column_suffix,summary_function):
    return dprep.SummaryColumnsValue(
                column_id=column_name,
                summary_column_name=column_name+'_'+column_suffix,
                summary_function=summary_function)

def generate_summary_columns(dflow):
    summary_columns = []
    for key in h_ts_dflow.get_profile().columns.keys():
        if key != 'DATETIME':
            summary_columns.append(generate_summary_column(key,'MAX',dprep.SummaryFunction.MAX))
            summary_columns.append(generate_summary_column(key,'MIN',dprep.SummaryFunction.MIN))
            summary_columns.append(generate_summary_column(key,'MEAN',dprep.SummaryFunction.MEAN))
            summary_columns.append(generate_summary_column(key,'MEDIAN',dprep.SummaryFunction.MEDIAN))
    return summary_columns

h_ts_summarized_dflow = h_ts_dflow.summarize(
    summary_columns=generate_summary_columns(h_ts_dflow),
    group_by_columns=['DATETIME'])

h_ts_summarized_dflow.head(4)

,DATETIME,h1_N1_MAX,h1_N1_MIN,h1_N1_MEAN,h1_N1_MEDIAN,h1_S1_MAX,h1_S1_MIN,h1_S1_MEAN,h1_S1_MEDIAN,h1_Z1_MAX,...,h2_N1_MEAN,h2_N1_MEDIAN,h2_S1_MAX,h2_S1_MIN,h2_S1_MEAN,h2_S1_MEDIAN,h3_CLOAD_MAX,h3_CLOAD_MIN,h3_CLOAD_MEAN,h3_CLOAD_MEDIAN
0,2012-01-01 01:00:00,2.789946,2.789946,2.789946,2.789946,-3.860854,-3.860854,-3.860854,-3.860854,-0.675159,...,214.201076,214.201076,76.939389,76.939389,76.939389,76.939389,7458.280255,7458.280255,7458.280255,7458.280255
1,2012-01-01 02:00:00,2.785634,2.785634,2.785634,2.785634,-3.788318,-3.788318,-3.788318,-3.788318,-0.675159,...,208.534389,208.534389,66.742051,66.742051,66.742051,66.742051,7214.649682,7214.649682,7214.649682,7214.649682
2,2012-01-01 03:00:00,2.786000,2.786000,2.786000,2.786000,-3.887990,-3.887990,-3.887990,-3.887990,-0.676115,...,188.653331,188.653331,70.460099,70.460099,70.460099,70.460099,7018.789809,7018.789809,7018.789809,7018.789809
3,2012-01-01 04:00:00,2.785965,2.785965,2.785965,2.785965,-4.119672,-4.119672,-4.119672,-4.119672,-0.700669,...,192.039484,192.039484,76.141580,76.141580,76.141580,76.141580,6912.420382,6912.420382,6912.420382,6912.420382


In [275]:
# =================
# dataprep.py (WIP)
# =================

h_ts_1_dflow = dprep.auto_read_file(DataPath(blob_store,'datasets/time-series/S_ACTUALS.csv'))
h_ts_2_dflow = dprep.auto_read_file(DataPath(blob_store,'datasets/time-series/W_ACTUALS.csv'))
h_ts_3_dflow = dprep.auto_read_file(DataPath(blob_store,'datasets/time-series/C_LOAD.csv'))
d_ts_1_dflow = dprep.auto_read_file(DataPath(blob_store,'datasets/time-series/X1.csv'))
d_ts_2_dflow = dprep.auto_read_file(DataPath(blob_store,'datasets/time-series/X2.csv'))

# Pivot data
h_ts_1_pivot_dflow = h_ts_1_dflow.pivot(['NODE_ID'],'MW',
                                        azureml.dataprep.api.engineapi.typedefinitions.SummaryFunction.MAX,
                                        ['MYDATE','HOUR'])
h_ts_2_pivot_dflow = h_ts_2_dflow.pivot(['NODE_ID'],'MW',
                                        azureml.dataprep.api.engineapi.typedefinitions.SummaryFunction.MAX,
                                        ['MYDATE','HOUR'])

# merge DATE and HOUR and update datatype for DATETIME column
def ts_merge_date_hour_to_datetime(dflow, date_column_name, hour_column_name):
    # merge columns
    dflow = dflow.derive_column_by_example(
        source_columns = [date_column_name,hour_column_name],
        new_column_name = 'DATETIME',
        example_data = [({date_column_name: '1/1/2012', hour_column_name: '1'},    '01/01/2012 01:00'),
                        ({date_column_name: '10/10/2012', hour_column_name: '15'}, '10/10/2012 15:00'),
                        ({date_column_name: '1/17/2012', hour_column_name: '12'},  '01/17/2012 12:00')]
        ).drop_columns([date_column_name,hour_column_name])
    # update data type
    builder = dflow.builders.set_column_types()
    builder.learn()
    builder.conversion_candidates['DATETIME'] = (dprep.FieldType.DATE, ['%d/%m/%Y %H:%M'])
    return builder.to_dataflow()

# generate all DATETIME columns with proper data type
h_ts_1_pivot_dt_dflow = ts_merge_date_hour_to_datetime(h_ts_1_pivot_dflow,'MYDATE','HOUR')
h_ts_2_pivot_dt_dflow = ts_merge_date_hour_to_datetime(h_ts_2_pivot_dflow,'MYDATE','HOUR')
h_ts_3_dt_dflow =       ts_merge_date_hour_to_datetime(h_ts_3_dflow,      'DATE','HE')

# JOINING DATA SET h_ts_dflow=join(h1,h2,h3)
h_ts_dflow = dprep.Dataflow.join(
    dprep.Dataflow.join(h_ts_1_pivot_dt_dflow,
                        h_ts_2_pivot_dt_dflow,
                        join_key_pairs=[('DATETIME', 'DATETIME')],
                        left_column_prefix='h1_',right_column_prefix='h2_'
                       ).drop_columns(['h2_DATETIME']).rename_columns({'h1_DATETIME':'DATETIME'}),
    h_ts_3_dt_dflow,
    join_key_pairs=[('DATETIME', 'DATETIME')],
    left_column_prefix='',
    right_column_prefix='h3_').drop_columns(['h3_DATETIME'])

# JOINING DATA SET d_ts_dflow=join(d1,d2)
d_ts_dflow = dprep.Dataflow.join(
    d_ts_1_dt_dflow,
    d_ts_2_dt_dflow,
    join_key_pairs=[('RDATE', 'RDATE')],
    left_column_prefix='',
    right_column_prefix='r_').drop_columns(['r_RDATE']).rename_columns({'r_X2':'X2'})

# helper: generate summary column
def generate_summary_column(column_name,column_suffix,summary_function):
    return dprep.SummaryColumnsValue(
                column_id=column_name,
                summary_column_name=column_name+'_'+column_suffix,
                summary_function=summary_function)

# helper: generate summary column for a few functions for each column that's not DATETIME
def generate_summary_columns(dflow):
    summary_columns = []
    for key in h_ts_dflow.get_profile().columns.keys():
        if key != 'DATETIME':
            summary_columns.append(generate_summary_column(key,'MAX',dprep.SummaryFunction.MAX))
            summary_columns.append(generate_summary_column(key,'MIN',dprep.SummaryFunction.MIN))
            summary_columns.append(generate_summary_column(key,'MEAN',dprep.SummaryFunction.MEAN))
            summary_columns.append(generate_summary_column(key,'MEDIAN',dprep.SummaryFunction.MEDIAN))
    return summary_columns

# summarize h_ts_dflow to daily
h_ts_summarized_dflow = h_ts_dflow.summarize(
    summary_columns=generate_summary_columns(h_ts_dflow),
    group_by_columns=['DATETIME'])

# join h and d series
training_dflow = dprep.Dataflow.join(
    h_ts_summarized_dflow,
    d_ts_dflow,
    join_key_pairs=[('DATETIME', 'RDATE')],
    left_column_prefix='',
    right_column_prefix='r_').drop_columns(['r_RDATE']).rename_columns({'r_X1':'X1','r_X2':'X2'})

# =============================
# end of dataprep.py code (WIP)
# =============================

In [277]:
training_dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
DATETIME,FieldType.DATE,2012-01-04 00:00:00+00:00,2016-02-02 00:00:00+00:00,494.0,0.0,494.0,0.0,0.0,0.0,494,,,,,,,,,,,,,,,,
h1_N1_MAX,FieldType.DECIMAL,-11.2282,173.853,494.0,0.0,494.0,0.0,0.0,0.0,494,-11.2282,-1.03639,-1.03898,-0.917647,-0.753907,-0.530348,2.51605,14.3176,173.853,0.0817778,8.21124,67.4245,19.3625,403.636,-0.0111603,-1.16208
h1_N1_MIN,FieldType.DECIMAL,-11.2282,173.853,494.0,0.0,494.0,0.0,0.0,0.0,494,-11.2282,-1.03639,-1.03898,-0.917647,-0.753907,-0.530348,2.51605,14.3176,173.853,0.0817778,8.21124,67.4245,19.3625,403.636,-0.0111603,-1.16208
h1_N1_MEAN,FieldType.DECIMAL,-11.2282,173.853,494.0,0.0,494.0,0.0,0.0,0.0,494,-11.2282,-1.03639,-1.03898,-0.917647,-0.753907,-0.530348,2.51605,14.3176,173.853,0.0817778,8.21124,67.4245,19.3625,403.636,-0.0111603,-1.16208
h1_N1_MEDIAN,FieldType.DECIMAL,-11.2282,173.853,494.0,0.0,494.0,0.0,0.0,0.0,494,-11.2282,-1.03639,-1.03898,-0.917647,-0.753907,-0.530348,2.51605,14.3176,173.853,0.0817778,8.21124,67.4245,19.3625,403.636,-0.0111603,-1.16208
h1_S1_MAX,FieldType.DECIMAL,-10.963,202.417,494.0,0.0,494.0,0.0,0.0,0.0,494,-10.963,-8.18442,-8.20619,-7.42491,-6.1581,-2.88146,1.96412,11.9707,202.417,-4.28139,10.5664,111.649,15.6945,296.134,3.1268,-10.963
h1_S1_MIN,FieldType.DECIMAL,-10.963,202.417,494.0,0.0,494.0,0.0,0.0,0.0,494,-10.963,-8.18442,-8.20619,-7.42491,-6.1581,-2.88146,1.96412,11.9707,202.417,-4.28139,10.5664,111.649,15.6945,296.134,3.1268,-10.963
h1_S1_MEAN,FieldType.DECIMAL,-10.963,202.417,494.0,0.0,494.0,0.0,0.0,0.0,494,-10.963,-8.18442,-8.20619,-7.42491,-6.1581,-2.88146,1.96412,11.9707,202.417,-4.28139,10.5664,111.649,15.6945,296.134,3.1268,-10.963
h1_S1_MEDIAN,FieldType.DECIMAL,-10.963,202.417,494.0,0.0,494.0,0.0,0.0,0.0,494,-10.963,-8.18442,-8.20619,-7.42491,-6.1581,-2.88146,1.96412,11.9707,202.417,-4.28139,10.5664,111.649,15.6945,296.134,3.1268,-10.963
h1_Z1_MAX,FieldType.DECIMAL,-1.83546,5.94535,494.0,0.0,494.0,0.0,0.0,0.0,492,-1.83546,-1.68655,-1.68947,-1.53338,-1.36637,-1.06828,1.38516,4.34993,5.94535,-0.994631,1.17549,1.38177,3.17753,10.8389,-0.494324,-1.83546


In [ ]:
# The best practice is to use separate folders for scripts and its dependent files
# for each step and specify that folder as the source_directory for the step.
# This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted).
# Since changes in any files in the source_directory would trigger a re-upload of the snapshot, this helps
# keep the reuse of the step when there are no changes in the source_directory of the step.
source_directory_dataprep = 'src/dataprep'
# Step
use_case_1_dataprep_sdk_step = PythonScriptStep(
    script_name="dataprep.py",
    arguments=["--output", d_use_case_1_dataprep_sdk_pd],
    inputs=[h_time_series_1_dr,
            h_time_series_2_dr,
            h_time_series_3_dr,
            d_time_series_1_dr,
            d_time_series_2_dr],
    outputs=[d_use_case_1_dataprep_sdk_pd],
    compute_target=aml_compute,
    source_directory=source_directory_dataprep,
    runconfig=run_config
)

print("== PythonScriptStep use_case_1_dataprep_step created")

### Build the pipeline and submit an Experiment run

In [ ]:
pipeline = Pipeline(workspace=ws, steps=[use_case_1_dataprep_sdk_step])
print ("== Pipeline is built")

In [ ]:
pipeline_run = Experiment(ws, 'use-case-1-dataprep-sdk').submit(pipeline)
print("== Pipeline is submitted for execution")

In [ ]:
RunDetails(pipeline_run).show()

### Wait for pipeline run to complete

In [ ]:
pipeline_run.wait_for_completion(show_output=True)

### See Outputs

See where outputs of each pipeline step are located on your datastore.

***Wait for pipeline run to complete, to make sure all the outputs are ready***

In [ ]:
# Get Steps
for step in pipeline_run.get_steps():
    print("== Outputs of step " + step.name)
    
    # Get a dictionary of StepRunOutputs with the output name as the key 
    output_dict = step.get_outputs()
    
    for name, output in output_dict.items():
        output_reference = output.get_port_data_reference() # Get output port data reference
        print("\tname: " + name)
        print("\tdatastore: " + output_reference.datastore_name)
        print("\tpath on datastore: " + output_reference.path_on_datastore)

In [ ]:
# REGISTER a new version of the final output as a Dataset

from azureml.core import Dataset, Datastore
from azureml.data.datapath import DataPath

# find output dataset
for step in pipeline_run.get_steps():
    output_dict = step.get_outputs()
    for name, output in output_dict.items():
        if name == 'd_use_case_1_dataprep_sdk':
            # generate a Tabular DataSet for it
            output_reference = output.get_port_data_reference()
            datastore_path = [DataPath(blob_store, output_reference.path_on_datastore)]
            ds = Dataset.Tabular.from_delimited_files(datastore_path)
            dataset_name = 'd_use_case_1_dataprep_sdk'
            ds.register(ws, name=dataset_name, create_new_version=True)
            print("== Registered new version of dataset: " + dataset_name)